In [1]:
import numpy as np
import pandas as pd



import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-ticks')
SMALL_SIZE = 13
MEDIUM_SIZE = 14
BIGGER_SIZE = 16



from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
df = pd.read_csv("")
df.head(10)
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns()

In [ ]:
df.drop(columns=[],axis=1)


In [ ]:
data = data.replace({'Not Available':np.nan})

for col in list(data.columns):
    if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in col 
        or 'therms' in col or 'gal' in col or 'Score' in col):
        data[col] = data[col].astype(float)

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)#concat on columns
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
missing_values_table(train_df)

In [ ]:
missing_df = missing_values_table(data)

missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('We will remove %d columns.' % len(missing_columns))

# Drop the columns
data = data.drop(columns = list(missing_columns))

# For older versions of pandas (https://github.com/pandas-dev/pandas/issues/19078)
# data = data.drop(list(missing_columns), axis = 1)

In [ ]:
# Histogram Plot of Site EUI
figsize(8, 8)
plt.hist(data['Site EUI (kBtu/ft²)'].dropna(), bins = 20, edgecolor = 'black');
plt.xlabel('Site EUI'); 
plt.ylabel('Count'); plt.title('Site EUI Distribution');


# Calculate first and third quartile
first_quartile = data['Site EUI (kBtu/ft²)'].describe()['25%']
third_quartile = data['Site EUI (kBtu/ft²)'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove outliers
data = data[(data['Site EUI (kBtu/ft²)'] > (first_quartile - 3 * iqr)) &
            (data['Site EUI (kBtu/ft²)'] < (third_quartile + 3 * iqr))]

In [ ]:
# Find all correlations and sort 
correlations_data = data.corr()['score'].sort_values()

# Print the most negative correlations
print(correlations_data.head(15), '\n')

# Print the most positive correlations
print(correlations_data.tail(15))


numeric_subset = data.select_dtypes('number')

# Create columns with square root and log of numeric columns
for col in numeric_subset.columns:
    # Skip the Energy Star Score column
    if col == 'score':
        next
    else:
        numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
        numeric_subset['log_' + col] = np.log(numeric_subset[col])

# Select the categorical columns
categorical_subset = data[['Borough', 'Largest Property Use Type']]

# One hot encode
categorical_subset = pd.get_dummies(categorical_subset)

# Join the two dataframes using concat
# Make sure to use axis = 1 to perform a column bind
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

# Drop buildings without an energy star score
features = features.dropna(subset = ['score'])

# Find correlations with the score 
correlations = features.corr()['score'].dropna().sort_values()

In [ ]:
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''
    
    # Dont want to remove correlations between Energy Star Score
    y = x['score']
    x = x.drop(columns = ['score'])
    
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                # print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    x = x.drop(columns = ['Weather Normalized Site EUI (kBtu/ft²)', 
                          'Water Use (All Water Sources) (kgal)',
                          'log_Water Use (All Water Sources) (kgal)',
                          'Largest Property Use Type - Gross Floor Area (ft²)'])
    
    # Add the score back in to the data
    x['score'] = y
               
    return x




# Remove the collinear features above a specified correlation coefficient
features = remove_collinear_features(features, 0.6)
# Remove any columns with all na values
features  = features.dropna(axis=1, how = 'all')
features.shape






# Separate out the features and targets
features = score.drop(columns='score')
targets = pd.DataFrame(score['score'])

# Replace the inf and -inf with nan (required for later imputation)
features = features.replace({np.inf: np.nan, -np.inf: np.nan})

# Split into 70% training and 30% testing set
X, X_test, y, y_test = train_test_split(features, targets, test_size = 0.3, random_state = 42)

print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

In [ ]:
def Groupby_OneCol_comp_plot(df, col, plt_style = 'seaborn-ticks',color_palette = 'coolwarm'):
    gr = pd.DataFrame()
    gr['{} No'.format(col)] = df.groupby(col).size()
    gr['{} Ratio'.format(col)] = np.round(gr['{} No'.format(col)].divide(gr['{} No'.format(col)].sum())*100,0)
    print('Total No. of {} {}'.format(col,gr['{} No'.format(col)].sum()))
    plt.style.use(plt_style)
    sns.set_palette(sns.color_palette(color_palette))
    fig = plt.figure()
    plt.axis('off')
    fig.add_subplot(121)
    
    ax = gr['{} No'.format(col)].plot(kind='bar',title='{} Counts'.format(col), figsize=(16,8), color=sns.color_palette())
    _ = plt.setp(ax.get_xticklabels(), rotation = 0)
    for p in ax.patches:
        ax.annotate(np.round(p.get_height(),decimals=2),(p.get_x()+p.get_width()/2.,p.get_height()), ha='center', 
                    va='center', xytext=(0,10), textcoords='offset points')
    ax.get_yaxis().set_ticks([])
    plt.xlabel('')

    fig.add_subplot(122)
    plt.axis('off')
    gr.loc[:,'{} Ratio'.format(col)].plot(kind='pie', 
                                          autopct='%1.1f%%',shadow=False,title='{} Ratio'.format(col),
                                          legend=False,labels=None);
    sns.despine(top=True, right=True, left=True, bottom = False);
    

In [ ]:
def plot(table, legloc='upper right', plt_style= 'seabotn-ticks', color_palette='dark',sorter=None, stacked=False,
        kind ='bar', percentage=True, custom_title=None, minimal=True,figsize=(19,10),width=0.7):
    grouped = table
    if percentage == True:
        grouped = np.round(grouped.divide(grouped['Total'],axis=0)*100,0)
    try:
        del grouped['Total']
    except:
        pass
    
    if sorter:
        grouped = grouped[sorter]
        
    plt.style.use(plt_style)
    sns.set_palette(sns.color_palette(color_palette))
    ax = grouped.plot(kind=kind, stacked=stacked, figsize=figsize, width=width)
    _ = plt.setp(ax.get_xticklabels(), rotation=0)
    plt.legend(loc = legloc)
    
    if percentage == True:
        for p in ax.patches:
            ax.annotate('{}%'.format(int(np.round(p.get_height(),decimals=2))),
                       (p.get_x()+p.get_width()/2. , p.get_height()), ha='center',
                        va = 'center', xytext=(0,10), textcoords = 'offset points')
    else:
        for p in ax.patches():
            ax.annotate(np.round(p.get_height(), decimals=2), (p.get_x()+p.get_width()/2.,p.get_height()), 
            ha='center', va='center', xytext=(0,10), textcoords='offset points')
    
    if minimal == True:
        ax.get_yaxis().set_ticks([])
        plt.xlabel('')
        sns.despine(top=True, right=True, left= True, bottom=False);
    else:
        pass
    
    plt.title(custom_title)

In [ ]:
def Groupby_TwoCol_Plot(df, col1, col2, legloc = 'upper right', plt_style = 'ggplot', 
                        color_palette = 'dark', sorter = None, stacked=False, kind='bar',
                       percentage=True, custom_title=None, minimal=True, figsize=(14,6), width=0.6):
    
    grouped = df.groupby([col2, col1]).size().unstack(col2)
    grouped['Total'] = grouped.sum(axis=1)
    #grouped = grouped.sort_values('Total', ascending = False)
    plot(grouped, legloc=legloc, plt_style = plt_style, color_palette = color_palette, sorter=sorter,
        stacked = stacked, kind = kind, percentage = percentage, custom_title = custom_title, minimal= minimal, 
         figsize = figsize, width = width)

In [ ]:
age_labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-80']
age_group_values = pd.cut(not_missing.Age, range(0,81,10), right=True, labels=age_labels)














import re
deck = {"A":1, "B":2, "C":3, "D":4, "E":5, "F":6, "G":7, "U":8}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)

train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)








data = [train_df, test_df]
for dataset in data:
    med = train_df['Age'].median()
    std = test_df['Age'].std()
    is_null = dataset['Age'].isnull().sum()
    rand_age = np.random.randint(med-std, med+std, size = is_null)
    age_slice = dataset['Age'].copy()#array of values
    age_slice[np.isnan(age_slice)] = rand_age
    dataset['Age'] = age_slice
    dataset['Age'] = dataset['Age'].astype(int)
    
train_df['Age'].isnull().sum()








train_df['Embarked'].describe()

com_val = 'S'
data = [train_df, test_df]
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(com_val)
    
    
    
    
    
    
    
    
train_df.info()


data = [train_df, test_df]

for dataset in data:

    dataset['Fare'] = dataset['Fare'].fillna(0)

    dataset['Fare'] = dataset['Fare'].astype(int)

data = [train_df, test_df]
titles = {'Mr':1,'Miss':2,'Mrs':3,'Master':4,'Rare':5}

for dataset in data:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.',expand = False)
    dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev',
                                                'Sir','Jonkheer','Dona'],'Rare')
    dataset['Title'] = dataset['Title'].replace(['Mlle','Ms'],'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme','Mrs')

title = pd.get_dummies(train_df['Title'])
train_df = pd.concat([train_df,title],axis=1)

title = pd.get_dummies(test_df['Title'])
test_df = pd.concat([test_df,title],axis=1)
#dataset['Title'] = dataset['Title'].map(titles)
#dataset['Title'] = dataset['Title'].fillna(0)
#dataset = dataset.drop(['Name'], axis=1)

train_df = train_df.drop(['Name','Title'],axis=1)
test_df = test_df.drop(['Name','Title'],axis=1)










data = [train_df, test_df]

for dataset in data:

    dataset['Age'] = dataset['Age'].astype(int)

    dataset.loc[dataset['Age'] <=11, 'Age'] = 0

    dataset.loc[(dataset['Age'] >11) & (dataset['Age'] <=18), 'Age'] = 1

    dataset.loc[(dataset['Age'] >18) & (dataset['Age'] <=22), 'Age'] = 2

    dataset.loc[(dataset['Age'] >22) & (dataset['Age'] <=27), 'Age'] = 3

    dataset.loc[(dataset['Age'] >27) & (dataset['Age'] <=33), 'Age'] = 4

    dataset.loc[(dataset['Age'] >33) & (dataset['Age'] <=40), 'Age'] = 5

    dataset.loc[dataset['Age'] >40, 'Age'] = 6

train_df['Age'].value_counts()
data = [train_df, test_df]
for dataset in data:
    dataset['Fare'] = dataset['Fare'].astype(int)
    dataset.loc[dataset['Fare'] <=7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] >7.91) & (dataset['Fare'] <=14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] >14.454) & (dataset['Fare'] <=31), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] >31) & (dataset['Fare'] <=99), 'Fare'] = 3
    dataset.loc[(dataset['Fare'] >99) & (dataset['Fare'] <=250), 'Fare'] = 4
    #dataset.loc[(dataset['Fare'] >33) & (dataset['Fare'] <=40), 'Fare'] = 5
    dataset.loc[dataset['Fare'] >250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)
train_df['Fare'].value_counts()


In [ ]:
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, X_train, Y_train, cv=10, scoring='accuracy')

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:",scores.std())